<a href="https://colab.research.google.com/github/Samreeneducation/thesis/blob/master/cyberbullying_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Detecting Cyberbullying in comments

In [0]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, make_scorer
from time import time

from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier, BaggingClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.externals import joblib

In [0]:
from google.colab import files
uploaded = files.upload()

In [0]:
df_scraped = pd.read_csv('data_test_clean.csv')


In [0]:
from google.colab import files
uploaded = files.upload()

In [0]:
df_public= pd.read_csv('datatrainclean.csv')

In [0]:
df_scraped.head(2)


In [0]:
df_public.head(2)

In [0]:
# for dropping any row from dataframe incase drop id from data frame
df_scraped.drop_duplicates(inplace = True)
df_scraped.drop('id',axis = 'columns', inplace = True)

df_public.drop_duplicates(inplace = True)

In [0]:
df = pd.concat([df_scraped, df_public])


In [0]:
df.shape


In [0]:
plt.figure(figsize = (7,7))
sorted_counts = df['toxic'].value_counts()
plt.pie(sorted_counts, labels= sorted_counts.index, startangle = 90, counterclock = False, wedgeprops = {'width' : 0.6},
       autopct='%1.2f%%', pctdistance = 0.7, textprops = {'color': 'black', 'fontsize' : 15}, shadow = True,
        colors = sns.color_palette("Paired")[7:])
plt.text(x = -0.35, y = 0, s = 'toxic: {}'.format(df.shape[0]))

plt.title('Distribution of toxic in the Dataset', fontsize = 16);

In [0]:
plt.figure(figsize = (7,7))
sorted_counts = df['severe_toxic'].value_counts()
plt.pie(sorted_counts, labels= sorted_counts.index, startangle = 90, counterclock = False, wedgeprops = {'width' : 0.6},
       autopct='%1.2f%%', pctdistance = 0.7, textprops = {'color': 'black', 'fontsize' : 15}, shadow = True,
        colors = sns.color_palette("Paired")[7:])
plt.text(x = -0.35, y = 0, s = 'severe_toxic: {}'.format(df.shape[0]))

plt.title('Distribution of severe_toxic in the Dataset', fontsize = 16);

In [0]:
plt.figure(figsize = (7,7))
sorted_counts = df['obscene'].value_counts()
plt.pie(sorted_counts, labels= sorted_counts.index, startangle = 90, counterclock = False, wedgeprops = {'width' : 0.6},
       autopct='%1.2f%%', pctdistance = 0.7, textprops = {'color': 'black', 'fontsize' : 15}, shadow = True,
        colors = sns.color_palette("Paired")[7:])
plt.text(x = -0.35, y = 0, s = 'obscene: {}'.format(df.shape[0]))

plt.title('Distribution of obscene in the Dataset', fontsize = 16);

In [0]:
plt.figure(figsize = (7,7))
sorted_counts = df['threat'].value_counts()
plt.pie(sorted_counts, labels= sorted_counts.index, startangle = 90, counterclock = False, wedgeprops = {'width' : 0.6},
       autopct='%1.2f%%', pctdistance = 0.7, textprops = {'color': 'black', 'fontsize' : 15}, shadow = True,
        colors = sns.color_palette("Paired")[7:])
plt.text(x = -0.35, y = 0, s = 'threat: {}'.format(df.shape[0]))

plt.title('Distribution of threat in the Dataset', fontsize = 16);

In [0]:
plt.figure(figsize = (7,7))
sorted_counts = df['insult'].value_counts()
plt.pie(sorted_counts, labels= sorted_counts.index, startangle = 90, counterclock = False, wedgeprops = {'width' : 0.6},
       autopct='%1.2f%%', pctdistance = 0.7, textprops = {'color': 'black', 'fontsize' : 15}, shadow = True,
        colors = sns.color_palette("Paired")[7:])
plt.text(x = -0.35, y = 0, s = 'insult: {}'.format(df.shape[0]))

plt.title('Distribution of insult in the Dataset', fontsize = 16);

In [0]:
plt.figure(figsize = (7,7))
sorted_counts = df['identity_hate'].value_counts()
plt.pie(sorted_counts, labels= sorted_counts.index, startangle = 90, counterclock = False, wedgeprops = {'width' : 0.6},
       autopct='%1.2f%%', pctdistance = 0.7, textprops = {'color': 'black', 'fontsize' : 15}, shadow = True,
        colors = sns.color_palette("Paired")[7:])
plt.text(x = -0.35, y = 0, s = 'identity_hate: {}'.format(df.shape[0]))

plt.title('Distribution of identity_hate in the Dataset', fontsize = 16);

In [0]:
df['cyberbully'] = df.cyberbully.map({'bully': 1, 'notbully': 0})



Implementing a training and predicting Pipeline

In [0]:
#Separate training and testing data:
X_train, X_test, y_train, y_test = train_test_split(df['clean_comment'], 
                                                    df['cyberbully'], 
                                                    random_state=42)

print('Number of rows in the total set: {}'.format(df.shape[0]))
print('Number of rows in the training set: {}'.format(X_train.shape[0]))
print('Number of rows in the test set: {}'.format(X_test.shape[0]))

since out puts are not being save in outide colab here am pasting it as text:
Number of rows in the total set: 159571
Number of rows in the training set: 119678
Number of rows in the test set: 39893

In [0]:
# Instantiate the CountVectorizer method
count_vector = CountVectorizer(stop_words = 'english', lowercase = True)

# Fit the training data and then return the matrix
training_data = count_vector.fit_transform(X_train)

# Transform testing data and return the matrix. Note we are not fitting the testing data into the CountVectorizer()
testing_data = count_vector.transform(X_test)

Implement Pipeline:



In [0]:
def pipeline(learner_list, X_train, y_train, X_test, y_test): 
    '''
    inputs:
       - learner: the learning algorithm to be trained and predicted on
       - X_train: features training set
       - y_train: income training set
       - X_test: features testing set
       - y_test: income testing set
    '''
    
    # Get length of Training Data:
    size = len(y_train)
    
    results = {}
    final_results = []
    
    for learner in learner_list:
        
        # Store the learner name:
        results['Algorithm'] = learner.__class__.__name__

        # Fit the learner:
        start = time() # Get start time
        print("Training {}".format(learner.__class__.__name__))
        learner = learner.fit(X_train, y_train)
        end = time() # Get end time

        # Store the training time
        results['Training Time'] = end - start

        start = time() # Get start time
        predictions_test = learner.predict(X_test)
        predictions_train = learner.predict(X_train)
        end = time() # Get end time

        # Store the prediction time
        results['Prediction Time'] = end - start

        # Compute the Accuracy on Test Set
        results['Accuracy: Test'] = accuracy_score(y_test, predictions_test)

        # Compute the Accuracy on Training Set
        results['Accuracy: Train'] = accuracy_score(y_train, predictions_train)

        # Compute the F1 Score on Test Set
        results['F1 Score: Test'] = f1_score(y_test, predictions_test)

        # Compute the F1 Score on Training Set
        results['F1 Score: Train'] = f1_score(y_train, predictions_train)

        # Compute the Precision on Test Set
        results['Precision: Test'] = precision_score(y_test, predictions_test)

        # Compute the Precision on Training Set
        results['Precision: Train'] = precision_score(y_train, predictions_train)

        # Compute the Recall on Test Set
        results['Recall: Test'] = recall_score(y_test, predictions_test)

        # Compute the Recall on Training Set
        results['Recall: Train'] = recall_score(y_train, predictions_train)

        # Success
        print("Training {} finished in {:.2f} sec".format(learner.__class__.__name__, results['Training Time']))
        print('----------------------------------------------------')
        
        final_results.append(results.copy())
    # Return a dataframe of the results
    return final_results

In [0]:
#Put Algorithms in Pipeline:
# make a list of models
models = [MultinomialNB(), DecisionTreeClassifier(), LinearSVC(), AdaBoostClassifier(), 
          RandomForestClassifier(), BaggingClassifier(),
         LogisticRegression(), SGDClassifier(), KNeighborsClassifier()]

In [0]:
re = pipeline(models, training_data, y_train, testing_data, y_test)
results = pd.DataFrame(re)
results = results.reindex(columns = ['Algorithm', 'Accuracy: Test', 'Precision: Test', 'Recall: Test', 'F1 Score: Test', 'Prediction Time',
                          'Accuracy: Train', 'Precision: Train', 'Recall: Train', 'F1 Score: Train', 'Training Time'])

In [0]:
import re

In [0]:
# for hndling missing vlues
def replace_missing_value(df, number_features):

    imputer = Imputer(strategy="median")
    df_num = df[number_features]
    imputer.fit(df_num)
    X = imputer.transform(df_num)
    res_def = pd.DataFrame(X, columns=df_num.columns)
    return res_def